## 資料集資訊

In [ ]:
os.listdir('/Users/chiehfang/Desktop/Cancer_Detection/histopathologic-cancer-detection')

In [ ]:
print(len(os.listdir('/Users/chiehfang/Desktop/Cancer_Detection/histopathologic-cancer-detection/train')))
print(len(os.listdir('/Users/chiehfang/Desktop/Cancer_Detection/histopathologic-cancer-detection/test')))

## 數據前處理

In [ ]:
df_data = pd.read_csv('/Users/chiehfang/Desktop/Histopathologic-cancer-detection/train_labels.csv')

df_data['label'].value_counts()

In [ ]:
df_tumor = df_data[df_data['label']==1]
df_tumor.shape

### 數據清洗函式

In [ ]:
def badimg_detect(filename):
    badimg = cv2.imread(filename)
    badimg = cv2.cvtColor(badimg, cv2.COLOR_BGR2GRAY)
    imgcenter = np.array(badimg[32:64,32:64])
    ret,thresh1 = cv2.threshold(imgcenter,180,255,cv2.THRESH_BINARY)
    white = np.where(thresh1[:]>250)
    whitenum = (white[0].shape)[0]
    whiteratio = whitenum/(32*32)
    return whiteratio

In [ ]:
tumor_list = list(df_tumor['id'])
badimgs = []
count = 0
for tumorimg in tumor_list:
    whiteratio = badimg_detect(os.path.join('/Users/chiehfang/Desktop/Cancer_Detection/histopathologic-cancer-detection-datasets-training/train',tumorimg+'.tif'))
    count+=1  
    if whiteratio >0.95:
        badimgs.append(tumorimg)   
print('total:',count)
print('badimg:',len(badimgs))

In [ ]:
# Creating a dataframe of all the training images

df_data = pd.read_csv('/Users/chiehfang/Desktop/Cancer_Detection/histopathologic-cancer-detection/train_labels.csv')

# removing this image because it caused a training error previously
df_data = df_data[df_data['id'] != 'dd6dfed324f9fcb6f93f46f32fc800f2ec196be2']

# removing this image because it's black
df_data = df_data[df_data['id'] != '9369c7278ec8bcc6c880d99194de09fc2bd4efbe']

# removing this image because it's center are empty
for badimg in badimgs:
    df_data = df_data[df_data['id'] != badimg]


print(df_data.shape)

In [ ]:
df_data['label'].value_counts()

In [ ]:
print(df_data.head())

## 建立使用資料集

In [ ]:
#### Create the Train and Validation Sets

df_0 = df_data[df_data['label']==0].sample(SAMPLE_SIZE,random_state=101)
df_1 = df_data[df_data['label']==1].sample(SAMPLE_SIZE,random_state=101)

# concat the dataframes
df_data = pd.concat([df_0, df_1], axis=0).reset_index(drop=True)
# shuffle
df_data = shuffle(df_data)

df_data['label'].value_counts()

In [ ]:
# Now, for the train-test split

# stratify=y creates a balanced validation set.
y = df_data['label']

df_train, df_val = train_test_split(df_data, test_size=0.10, random_state=101, stratify=y)

print(df_train.shape)
print(df_val.shape)

### 資料存放結構

In [ ]:
# Create a new directory so that we will be using the ImageDataGenerator
base_dir='/Users/chiehfang/Desktop/Cancer_Detection/histopathologic-cancer-detection/base_dir'
os.mkdir(base_dir)

train_dir = os.path.join(base_dir, 'train_dir')
os.mkdir(train_dir)
no_tumor_tissue = os.path.join(train_dir, 'a_no_tumor_tissue')
os.mkdir(no_tumor_tissue)
has_tumor_tissue = os.path.join(train_dir, 'b_has_tumor_tissue')
os.mkdir(has_tumor_tissue)


val_dir = os.path.join(base_dir, 'val_dir')
os.mkdir(val_dir)
no_tumor_tissue = os.path.join(val_dir, 'a_no_tumor_tissue')
os.mkdir(no_tumor_tissue)
has_tumor_tissue = os.path.join(val_dir, 'b_has_tumor_tissue')
os.mkdir(has_tumor_tissue)

In [ ]:
# check that the folders have been created
os.listdir('/Users/chiehfang/Desktop/Cancer_Detection/histopathologic-cancer-detection/base_dir/train_dir')

In [ ]:
# Set the id as the index in df_data
df_data.set_index('id', inplace=True)

In [ ]:
# Get a list of train and val images
train_list = list(df_train['id'])

countt=1

# Transfer the train images

for image in train_list:
    
    # the id in the csv file does not have the .tif extension therefore we add it here
    fname = image + '.tif'
    # get the label for a certain image
    target = df_data.loc[image,'label']
    
    if countt %1000 ==0:
        print(countt)
    # these must match the folder names
    if target == 0:
        label = 'a_no_tumor_tissue'
    if target == 1:
        label = 'b_has_tumor_tissue'
    
    # source path to image
    src = os.path.join('/Users/chiehfang/Desktop/Cancer_Detection/histopathologic-cancer-detection/train', fname)
    img = cv2.imread(src,cv2.IMREAD_COLOR)
    cropimg = img[32:64,32:64]
    
    # destination path to image
    dst = os.path.join(train_dir, label, fname)
    cv2.imwrite(dst,cv2.cvtColor(cropimg, cv2.COLOR_BGR2RGB))
    
    countt=countt+1

In [ ]:
val_list = list(df_val['id'])
# Transfer the val images
count=1
for image in val_list:
    
    # the id in the csv file does not have the .tif extension therefore we add it here
    fname = image + '.tif'
    # get the label for a certain image
    target = df_data.loc[image,'label']
    
    if count %1000 ==0:
        print(count)
    
    # these must match the folder names
    if target == 0:
        label = 'a_no_tumor_tissue'
    if target == 1:
        label = 'b_has_tumor_tissue'
    
    # source path to image
    src = os.path.join('/Users/chiehfang/Desktop/Cancer_Detection/histopathologic-cancer-detection/train', fname)
    img = cv2.imread(src,cv2.IMREAD_COLOR)
    cropimg = img[32:64,32:64]
    
    # destination path to image
    dst = os.path.join(val_dir, label, fname)
    cv2.imwrite(dst,cv2.cvtColor(cropimg, cv2.COLOR_BGR2RGB))

    count=count+1

### 處理後資料集資訊

In [ ]:
# check how many train images we have in each folder

print(len(os.listdir('/Users/chiehfang/Desktop/Cancer_Detection/histopathologic-cancer-detection/base_dir_v1/train_dir/a_no_tumor_tissue')))
print(len(os.listdir('/Users/chiehfang/Desktop/Cancer_Detection/histopathologic-cancer-detection/base_dir_v1/train_dir/b_has_tumor_tissue')))

In [ ]:
# check how many val images we have in each folder

print(len(os.listdir('/Users/chiehfang/Desktop/Cancer_Detection/histopathologic-cancer-detection/base_dir_v1/val_dir/a_no_tumor_tissue')))
print(len(os.listdir('/Users/chiehfang/Desktop/Cancer_Detection/histopathologic-cancer-detection/base_dir_v1/val_dir/b_has_tumor_tissue')))